In [2]:
import os
RELSFILE = '.\\data\\20220801.as-rel.txt'
PATHSLISTFILE = '20220801.gained_from_5511.txt'
SECONDTARGET = '211181'
TARGET = 5511

In [3]:
# get gained and lost ASN lists
CONESFILE1 = '.\\data\\20220601.ppdc-ases.txt'
CONESFILE2 = '.\\data\\20220801.ppdc-ases.txt'

# find target cone in each of the cone files

current_line = '0 0'
if os.path.isfile(CONESFILE1):
    cdata = open(CONESFILE1, 'r', encoding='utf-8')
    # print(type(cdata1))
    cdata.readline()
    cdata.readline()
    while current_line[0:current_line.index(' ')] != str(TARGET):
        current_line = cdata.readline()
        if len(current_line) == 0:
            print('didn\'t find it')
            break
    # print(current_line)
else:
    print('bad filename (cones file 1)')

tcone1 = current_line[(current_line.index(' ')+1):len(current_line)]

current_line = '0 0'
if os.path.isfile(CONESFILE2):
    cdata = open(CONESFILE2, 'r', encoding='utf-8')
    # print(type(cdata1))
    cdata.readline()
    cdata.readline()
    while current_line[0:current_line.index(' ')] != str(TARGET):
        current_line = cdata.readline()
        if len(current_line) == 0:
            print('didn\'t find it')
            break
    # print(current_line)
else:
    print('bad filename (cones file 2)')

tcone2 = current_line[(current_line.index(' ')+1):len(current_line)]

# convert target cones into dicts

tc1d = {}
for asn in tcone1.split(' '):
    tc1d[int(asn)] = True

tc2d = {}
for asn in tcone2.split(' '):
    tc2d[int(asn)] = True
    
    # find lost and gained ASes (requires dict conversion)

lost_asns = []
gained_asns = []

for asn in tc1d:
    if not int(asn) in tc2d:
        lost_asns.append(int(asn))

for asn in tc2d:
    if not int(asn) in tc1d:
        gained_asns.append(int(asn))

In [4]:
# use paths list to make dict of pertinent ASNs

pertinent_asns = {}
with open(PATHSLISTFILE, 'r', encoding='utf-8') as pathslist:
    current_line = pathslist.readline()
    while len(current_line) != 0:
        for asn in current_line.split('|'):
            pertinent_asns[int(asn)] = True
        current_line = pathslist.readline()

# iterate through relationships file, extracting only relationships with pertinent ASes
# and storing them in a dict

relsdict = {}
with open(RELSFILE, 'r', encoding='utf-8') as rels:
    current_line = '#'
    while current_line[0] == '#':
        current_line = rels.readline()
    while len(current_line) != 0:
        clsplit = current_line.split('|')
        for i in range(3):
            clsplit[i] = int(clsplit[i])
        if clsplit[0] in pertinent_asns and clsplit[1] in pertinent_asns:
            if not clsplit[0] in relsdict:
                relsdict[clsplit[0]] = { clsplit[1]: clsplit[2] }
            else:
                relsdict[clsplit[0]][clsplit[1]] = clsplit[2]
        current_line = rels.readline()

In [5]:
# create a list of "annotated" paths, showing the relationships between the ASes

annotated_paths = []
with open(PATHSLISTFILE, 'r', encoding='utf-8') as pathslist:
    current_line = pathslist.readline()
    while len(current_line) != 0:
        clsplit = current_line.split('|')
        for i in range(len(clsplit)):
            clsplit[i] = int(clsplit[i])
        annotate = [clsplit[0]]
        for i in range(1, len(clsplit)):
            if clsplit[i-1] in relsdict and clsplit[i] in relsdict[clsplit[i-1]]:
                if relsdict[clsplit[i-1]][clsplit[i]] == -1:
                    annotate.append('<')
                elif relsdict[clsplit[i-1]][clsplit[i]] == 0:
                    annotate.append('-')
                else:
                    annotate.append('>')
            else:
                if clsplit[i] in relsdict:
                    if not clsplit[i-1] in relsdict[clsplit[i]]:
                        annotate.append(['?', clsplit[i]])
                        continue
                    if relsdict[clsplit[i]][clsplit[i-1]] == -1:
                        annotate.append('>')
                    elif relsdict[clsplit[i]][clsplit[i-1]] == 0:
                        annotate.append('-')
                    else:
                        annotate.append('<')
                else:
                    annotate.append('?')
            annotate.append(clsplit[i])
        # print(annotate)
        annotated_paths.append(annotate)
        current_line = pathslist.readline()

In [ ]:
# same as the one above, but only for paths that have the secondary target (SECONDTARGET)

annotated_paths = []
with open(PATHSLISTFILE, 'r', encoding='utf-8') as pathslist:
    current_line = pathslist.readline()
    while len(current_line) != 0:
        clsplit = current_line.split('|')
        if clsplit.count(SECONDTARGET) > 0:
            for i in range(len(clsplit)):
                clsplit[i] = int(clsplit[i])
            annotate = str(clsplit[0])
            for i in range(1, len(clsplit)):
                if clsplit[i-1] in relsdict and clsplit[i] in relsdict[clsplit[i-1]]:
                    if relsdict[clsplit[i-1]][clsplit[i]] == -1:
                        annotate = annotate + ' < '
                    elif relsdict[clsplit[i-1]][clsplit[i]] == 0:
                        annotate = annotate + ' - '
                    else:
                        annotate = annotate + ' > '
                else:
                    if clsplit[i] in relsdict:
                        if not clsplit[i-1] in relsdict[clsplit[i]]:
                            annotate = annotate + ' ? ' + str(clsplit[i])
                            continue
                        if relsdict[clsplit[i]][clsplit[i-1]] == -1:
                            annotate = annotate + ' > '
                        elif relsdict[clsplit[i]][clsplit[i-1]] == 0:
                            annotate = annotate + ' - '
                        else:
                            annotate = annotate + ' < '
                    else:
                        annotate = annotate + ' ? '
                annotate = annotate + str(clsplit[i])
            # print(annotate)
            annotated_paths.append(annotate.split(' '))
        current_line = pathslist.readline()

In [6]:
# start analyzing the paths by cropping them twice:
# first, to only the part after (inclusive) the first provider-customer relationship
# second, to only the part after (exclusive) the target

cropped_paths = []
for path in annotated_paths:
    new_path = []
    if path.count('<') != 0:
        new_path = path[(path.index('<') - 1):]
    if path.count(TARGET) != 0:
        new_path = path[(path.index(TARGET) + 2)::2]
        cropped_paths.append(new_path)

In [13]:
# go through the cropped paths and for each AS, create a list of every unique gained AS that comes after it

ct = 0
countstep = int(len(cropped_paths) / 10)
cones = {}
for path in cropped_paths:
    ct += 1
    if ct % countstep == 0:
        print('Progress: {}%'.format(int(100 * ct / len(cropped_paths) + 0.99999)))
    for i in range(len(path) - 1):
        if not path[i] in cones:
            cones[path[i]] = []
        for asn in path[i+1:]:
            if gained_asns.count(asn) != 0 and cones[path[i]].count(asn) == 0:
                cones[path[i]].append(asn)

Progress: 10%
Progress: 20%
Progress: 30%
Progress: 40%
Progress: 50%
Progress: 60%
Progress: 70%
Progress: 80%
Progress: 90%
Progress: 100%


In [14]:
for asn in cones:
    cones[asn] = len(cones[asn])

n = 0
for frequency, count in sorted(cones.items(), key=lambda f_c: f_c[1], reverse=True):
    print(frequency, count)
    n += 1
    if n > 100:
        break

12389 1904
3216 1804
13786 205
29076 162
35598 142
29226 124
201776 107
57344 104
49666 102
24785 91
174 89
60280 80
5617 76
49063 70
8595 67
202984 66
28329 63
6762 63
25478 62
12695 62
60068 60
28910 60
6663 58
50509 58
199599 58
60330 54
52091 53
48276 51
41798 50
45629 50
41689 49
43754 49
12406 44
9198 44
6453 43
29049 40
28761 40
35000 40
6697 39
25227 38
35168 37
9304 36
52468 35
60299 34
1299 34
3491 32
43201 32
30823 32
35104 32
29535 32
6789 31
8470 30
8492 29
48159 28
43727 28
6774 28
48347 27
59796 27
3356 27
12880 25
25184 25
41039 25
58224 25
2914 24
37662 24
262847 24
55330 24
35320 23
24739 23
25408 22
7693 21
44539 21
42337 20
12956 20
12713 20
196695 20
51346 20
49981 18
196925 18
264997 18
199995 18
9009 17
39533 17
34549 17
262596 17
30844 17
31430 17
9049 16
25549 16
29119 16
8953 16
37179 16
34718 15
39238 15
48166 15
58299 14
34123 14
42313 14
44843 14
61884 14
12773 14


In [15]:
def getcount(pair):
    return pair[1]

n = 0
for frequency, count in sorted(cones.items(), key=getcount, reverse=True):
    print(frequency, count)
    n += 1
    if n > 50:
        break

12389 1904
3216 1804
13786 205
29076 162
35598 142
29226 124
201776 107
57344 104
49666 102
24785 91
174 89
60280 80
5617 76
49063 70
8595 67
202984 66
28329 63
6762 63
25478 62
12695 62
60068 60
28910 60
6663 58
50509 58
199599 58
60330 54
52091 53
48276 51
41798 50
45629 50
41689 49
43754 49
12406 44
9198 44
6453 43
29049 40
28761 40
35000 40
6697 39
25227 38
35168 37
9304 36
52468 35
60299 34
1299 34
3491 32
43201 32
30823 32
35104 32
29535 32
6789 31
